# Эксперимент №2. SpanBERT+BERT-Chinese 

За основу берем скрипты [BERT for Coreference Resolution](https://github.com/mandarjoshi90/coref), которые обучаются на СoNLL2012, они были немного изменены, т.к. не работают с китайским языком.

В ходе экспериментов были протестировано несколько предобученных BERT-моделей. Наилучший результат показала модель [Chinese-BERT-wwm](https://github.com/ymcui/Chinese-BERT-wwm), поэтому используем её. 

Сама модель - это Берт + несколько слоёв. Тип модели: [e2e-coref model](https://github.com/kentonl/e2e-coref).

Структура модели (последний слой Берта и слои для определения спанов)
```
  name = bert/pooler/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
  name = span_width_embeddings:0, shape = (30, 20)
  name = mention_word_attn/output_weights:0, shape = (768, 1)
  name = mention_word_attn/output_bias:0, shape = (1,)
  name = mention_scores/hidden_weights_0:0, shape = (2324, 3000)
  name = mention_scores/hidden_bias_0:0, shape = (3000,)
  name = mention_scores/output_weights:0, shape = (3000, 1)
  name = mention_scores/output_bias:0, shape = (1,)
  name = span_width_prior_embeddings:0, shape = (30, 20)
  name = width_scores/hidden_weights_0:0, shape = (20, 3000)
  name = width_scores/hidden_bias_0:0, shape = (3000,)
  name = width_scores/output_weights:0, shape = (3000, 1)
  name = width_scores/output_bias:0, shape = (1,)
  name = genre_embeddings:0, shape = (7, 20)
  name = src_projection/output_weights:0, shape = (2324, 2324)
  name = src_projection/output_bias:0, shape = (2324,)
  name = antecedent_distance_emb:0, shape = (10, 20)
  name = output_weights:0, shape = (20, 1)
  name = output_bias:0, shape = (1,)
  name = coref_layer/same_speaker_emb:0, shape = (2, 20)
  name = coref_layer/antecedent_distance_emb:0, shape = (10, 20)
  name = coref_layer/segment_distance/segment_distance_embeddings:0, shape = (11, 20)
  name = coref_layer/slow_antecedent_scores/hidden_weights_0:0, shape = (7052, 3000)
  name = coref_layer/slow_antecedent_scores/hidden_bias_0:0, shape = (3000,)
  name = coref_layer/slow_antecedent_scores/output_weights:0, shape = (3000, 1)
  name = coref_layer/slow_antecedent_scores/output_bias:0, shape = (1,)
  name = coref_layer/f/output_weights:0, shape = (4648, 2324)
  name = coref_layer/f/output_bias:0, shape = (2324,)
```


Папка для сохранения всех данных - генерации датасетов, сохранения логов, сохранения обученной модели.

In [0]:
!mkdir data

Копируем OntoNotes 5.0:

In [0]:
!cp '/content/drive/My Drive/colab_data/Coreference/span_bert/data/ontonotes-release-5.0_LDC2013T19.tgz' ./

In [0]:
!tar xvzf /content/ontonotes-release-5.0_LDC2013T19.tgz

In [0]:
!git clone https://github.com/blanchefort/coref.git

Cloning into 'coref'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 741 (delta 3), reused 6 (delta 2), pack-reused 731
Receiving objects: 100% (741/741), 4.18 MiB | 21.08 MiB/s, done.
Resolving deltas: 100% (444/444), done.


Переходим в папку скачанных скриптов:

In [0]:
%cd coref

/content/coref


Устанавливаем зависимости:

In [0]:
!cat requirements.txt | sed 's/MarkupSafe==1.0/MarkupSafe==1.1.1/' > tmp
!mv tmp requirements.txt

In [0]:
%tensorflow_version 2.x
!pip uninstall -y tensorflow
!pip install -r requirements.txt --log install-log.txt -q

In [0]:
!./setup_all.sh

Генерируем датасеты из OntoNotes:

In [0]:
! ./setup_training.sh '/content/ontonotes-release-5.0' '/content/data' > /content/setup_log.txt

--2020-05-12 14:15:02--  http://conll.cemantix.org/2012/download/conll-2012-train.v4.tar.gz
Resolving conll.cemantix.org (conll.cemantix.org)... 66.147.244.155
Connecting to conll.cemantix.org (conll.cemantix.org)|66.147.244.155|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 71105451 (68M) [application/x-gzip]
Saving to: ‘/content/data/conll-2012-train.v4.tar.gz’

conll-2012-train.v4 100%[===================>]  67.81M  6.62MB/s    in 8.9s    

2020-05-12 14:15:11 (7.60 MB/s) - ‘/content/data/conll-2012-train.v4.tar.gz’ saved [71105451/71105451]

--2020-05-12 14:15:16--  http://conll.cemantix.org/2012/download/conll-2012-development.v4.tar.gz
Resolving conll.cemantix.org (conll.cemantix.org)... 66.147.244.155
Connecting to conll.cemantix.org (conll.cemantix.org)|66.147.244.155|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9270299 (8.8M) [application/x-gzip]
Saving to: ‘/content/data/conll-2012-development.v4.tar.gz’

conll-2012-develo

Генерируем файлы для обучения (получаются JSON-файлы). Важно указать правильный словарь. Нужно использовать именно словарь выбранной модели, т.к. другие словари дают плохое качество для китайского языка. Словарь всегда идёт вместе с предобученной моделью.

In [0]:
!python ./minimize.py /content/data/chinese_L-12_H-768_A-12/vocab.txt /content/data /content/data true

W0512 14:48:56.707460 140240500500352 deprecation_wrapper.py:119] From /content/coref/coref_ops.py:11: The name tf.NotDifferentiable is deprecated. Please use tf.no_gradient instead.

W0512 14:48:56.734206 140240500500352 deprecation_wrapper.py:119] From /content/coref/bert/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0512 14:48:58.872298 140240500500352 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

True
W0512 14:48:58.872742 140240500500352 deprecation_wrapper.py:119] From /content/coref/bert/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile inste

In [0]:
!cp /content/data/*.jsonlines '/content/drive/My Drive/colab_data/Coreference/span_bert/'

In [0]:
#!rm -rf /content/data/*

In [0]:
%cd ..

/content


In [0]:
!cp '/content/drive/My Drive/colab_data/Coreference/chinese_wwm_L-12_H-768_A-12.zip' ./

In [0]:
!unzip '/content/chinese_wwm_L-12_H-768_A-12.zip'

Archive:  /content/multi_cased_L-12_H-768_A-12.zip
   creating: multi_cased_L-12_H-768_A-12/
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: multi_cased_L-12_H-768_A-12/vocab.txt  
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: multi_cased_L-12_H-768_A-12/bert_config.json  


In [0]:
!cp /content/publish/* /content/data/chinese_L-12_H-768_A-12

In [0]:
!mkdir /content/data/bert_base/ 

In [0]:
!cp /content/publish/* /content/data/bert_base

In [0]:
%cd /content/coref

/content/coref


Запускаем обучение:

In [0]:
!GPU=0 data_dir=/content/data python ./train.py train_bert_base

Evaluated 251/252 examples.
Average F1 (py): 64.63% on 252 docs
Average precision (py): 69.78%
Average recall (py): 60.19%
I0512 22:21:31.760513 140199826343808 train.py:73] [34000] evaL_f1=0.6463, max_f1=0.6463
I0512 22:22:30.854428 140199826343808 train.py:58] [34100] loss=22.42, steps/s=1.48
I0512 22:23:26.650951 140199826343808 train.py:58] [34200] loss=14.06, steps/s=1.48
I0512 22:24:20.985370 140199826343808 train.py:58] [34300] loss=22.28, steps/s=1.48
I0512 22:25:26.151252 140199826343808 train.py:58] [34400] loss=31.15, steps/s=1.48
I0512 22:26:22.267604 140199826343808 train.py:58] [34500] loss=21.83, steps/s=1.48
I0512 22:27:27.142749 140199826343808 train.py:58] [34600] loss=21.71, steps/s=1.48
I0512 22:28:24.843075 140199826343808 train.py:58] [34700] loss=14.16, steps/s=1.48
I0512 22:29:22.795822 140199826343808 train.py:58] [34800] loss=17.68, steps/s=1.48
I0512 22:30:20.795475 140199826343808 train.py:58] [34900] loss=17.70, steps/s=1.48
I0512 22:31:20.923572 1401998263

Лучший результат после 57 эпох обучения:

* Лучшая эпоха: 46
* Average F1 (py): 65.40%
* Average precision (py): 68.47%
* Average recall (py): 65.40%


Гиперпараметры обучения:

`/coref/experiments.conf`

```
# Computation limits.
  max_top_antecedents = 50
  max_training_sentences = 5
  top_span_ratio = 0.4
  max_num_speakers = 20
  max_segment_len = 256

  # Learning
  bert_learning_rate = 1e-5
  task_learning_rate = 2e-4
  num_docs = 2802

  # Model hyperparameters.
  dropout_rate = 0.3
  ffnn_size = 1000
  ffnn_depth = 1
  num_epochs = 20
  feature_size = 20
  max_span_width = 30
  use_metadata = true
  use_features = true
  use_segment_distance = true
  model_heads = true
  coref_depth = 2
  coarse_to_fine = true
  fine_grained = true
  use_prior = true

  adam_eps = 1e-6
  task_optimizer = adam
```

Логи и обученная модель расположены в папке: `data/train_bert_base`

In [0]:
# Save best result
!cp -rf /content/data/train_bert_base/ '/content/drive/My Drive/colab_data/Coreference/span_bert/models/'

In [0]:
!ls /content/data/train_bert_base

checkpoint				     model-57000.index
events.out.tfevents.1589299071.89c75a8806eb  model-57000.meta
model-56000.data-00000-of-00001		     model.max.ckpt.data-00000-of-00001
model-56000.index			     model.max.ckpt.index
model-56000.meta			     stdout.log
model-57000.data-00000-of-00001


In [0]:
!pip install tensorboardX

     |████████████████████████████████| 204kB 4.9MB/s 


In [0]:
#%load_ext tensorboard

In [0]:
#%tensorboard --logdir /content/data/train_bert_base